In [1]:
import numpy as np
import pandas as pd
import sys
from sklearn import ensemble as em
train = pd.read_csv('year1-data.csv')
train2  = pd.read_csv('year2-data.csv')

X_test = pd.read_csv('testing_data.csv')
#train.shape #6588(366*18)*26
#train2.shape #6570(365*18)*26
#X_test.shape #9000*11

In [2]:
#train = train.replace("#","")
#train = train.replace("\*","")
#train = train.replace("x","")
def preprocess(train):
    train.replace("#",'',inplace=True,regex=True)
    train.replace("x",'',inplace=True,regex=True)
    train.replace("\*",'',inplace=True,regex=True)
    train.replace("NR",0,inplace=True) #將NR變成0
    train.fillna(0, inplace=True) #把NAN變成0
    train = train.iloc[:,2:]
    return train

In [3]:
train =  preprocess(train) #6588*24
train2 = preprocess(train2) #6570*24
X_test = preprocess(X_test) #9000*9


In [4]:
def get_feature(train,test=False):
    train = np.array(train,dtype=float)# to array 6588*24
    tmp=[]
    feature = []
    for i  in range(train.shape[0]): #0~6587
        index = i%18
        tmp.append(train[i])
        if index==17:
            tmp = np.array(tmp,dtype=float)#shape7*24
            feature.append(tmp)
            tmp = []
    feature = np.array(feature,dtype=float)#366*168
    print(feature.shape)
    return feature

In [5]:
train = get_feature(train)#shape 366*168
train2 = get_feature(train2)#shape 365*168
X_test = get_feature(X_test)#500*7*9


(366, 18, 24)
(365, 18, 24)
(500, 18, 9)


In [6]:
train = np.concatenate((train, train2), axis=0)#shape 731*7*24

In [7]:
tmp = train[0]
tmp.shape
for i in range(1,train.shape[0]):
    tmp =  np.concatenate((tmp, train[i]), axis=1)
train = tmp #7*17544

In [8]:
tmp = []
X_test.shape
for i in range(X_test.shape[0]):
    tmp.append(X_test[i].flatten())
X_test = tmp 
X_test = np.array(X_test,dtype=float)#shape 500*63
X_test.shape


(500, 162)

In [9]:
def valid(x, y):
    if y <= 2:
        y = 4
    if y>100:
        y = 80
    for i in range(9):
        if x[9,i] <= 2:
            x[9,i] = 4
        if x[9,i] > 100:
            x[9,i] = 80

In [10]:
X=[]
Y=[]
for i in range(0, train.shape[1]-9):
    x =  train[:, i:i+9]
    y = train[9, i+9]
    valid(x,y)
    X.append(x.flatten())
    Y.append(y)

X = np.array(X)#shape 5845*63
Y = np.array(Y)#shape 5845*1
X.shape
#Y.shape

(17535, 162)

In [11]:
gbmodel = em.GradientBoostingRegressor(verbose = True,n_estimators=4000, learning_rate=0.01,max_depth=10, loss='ls').fit(X,Y)
ans = gbmodel.predict(X_test)


      Iter       Train Loss   Remaining Time 
         1        4853.1214           49.45m
         2        4762.0060           50.10m
         3        4672.7079           49.32m
         4        4585.1843           49.03m
         5        4499.3973           48.83m
         6        4415.3158           48.84m
         7        4332.9073           49.44m
         8        4252.1385           49.49m
         9        4172.9770           49.59m
        10        4095.3911           49.68m
        20        3402.5438           49.47m
        30        2832.5539           49.32m
        40        2343.6776           49.35m
        50        1951.1967           49.63m
        60        1622.4701           49.78m
        70        1350.3014           49.41m
        80        1128.3279           49.04m
        90         947.7192           48.98m
       100         796.0798           48.89m
       200         168.8611           47.88m
       300          57.2769           46.89m
       40

In [13]:
#predict = model.predict(X_test)
for i in range(len(ans)):
    if ans[i]<0:
        ans[i] = 19.445

In [14]:
 with open('gradient_bootsing_result.csv', 'w') as f:
        print('id,value', file=f)
        for (i, p) in enumerate(ans) :
            print('id_{},{}'.format(i, p), file=f)

# TODO #

* ~~去掉七八月~~
* ~~shift 3 小時~~
* ~~只做PM2.5 PM10 NO2影響大......~~
* ~~負的資料砍掉~~
* n-fold
* normalize
* gradient bootsing

